In [14]:
%load_ext autoreload
%autoreload 2

from transactions import transactions
from dividends import dividends, dbs_promotes
from stocks import stocks

open_positions = {}
for t in transactions:
    if t['tick'] in open_positions:
        o_p = open_positions[t['tick']]
        o_p['transactions'].append(t)
    else:
        o_p = {'transactions': [t], 'total_cost':0, 'total_quantity':0, 'avg_cost':0, \
               'dividends':[], 'total_dividends':0, 'current_price':0, \
               'price_change':0, 'pl':0, 'pl_percent':0}
        open_positions[t['tick']] = o_p

for d in dividends:
    open_positions[d['tick']]['dividends'].append(d)
        
def summerize_position(p):
    for t in p['transactions']:
        p['total_cost'] += (t['buy_price']*t['quantity']+t['commission'])
        p['total_quantity'] += t['quantity']
        p['avg_cost'] = p['total_cost']/p['total_quantity']
    for d in p['dividends']:
        p['total_dividends'] += d['amount']
        
print('Positions')
for k, v in open_positions.items():
    summerize_position(v)
    print(k + ' average cost: ' + str(v['avg_cost']) + ', dividends: ' + str(v['total_dividends']))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Positions
D05.SI average cost: 20.43515714285714, dividends: 126
ME8U.SI average cost: 3.050636, dividends: 286.8
AJBU.SI average cost: 2.8647714285714287, dividends: 134.26
C2PU.SI average cost: 4.167125, dividends: 142.2
BABA average cost: 246.11894505494507, dividends: -1.87
AAPL average cost: 114.22064516129032, dividends: 9.24
MSFT average cost: 208.8764705882353, dividends: 9.05
NVDA average cost: 535.34, dividends: 0
ARKK average cost: 103.88905263157893, dividends: 108.14
SPY average cost: 345.78545454545457, dividends: 20.05
ARKW average cost: 119.82363636363635, dividends: 39.48
ARKG average cost: 76.22421875, dividends: 56.37
ARKQ average cost: 73.97458333333334, dividends: 27.02
ARKF average cost: 50.910625, dividends: 6.24
FB average cost: 260.9233333333333, dividends: 0
PRNT average cost: 44.11103448275862, dividends: 0
TSLA average cost: 722.8345454545455, dividends: 0
SOXX average co

In [15]:
from pandas_datareader import data as wb
import os

for s in open_positions:
    d = wb.DataReader(s, data_source='yahoo', start='1997-1-1')
    file_path = os.path.join('data', s+'.csv')
    d.to_csv(file_path)

In [16]:
import pandas as pd

def load_data(tick):
    file_path = os.path.join('data', tick+'.csv')
    data = pd.read_csv(file_path)
    dt_idx = pd.DatetimeIndex(data['Date'])
    data.index = dt_idx

    return data

for k, v in open_positions.items():
    d = load_data(k)
    v['current_price'] = d['Adj Close'].iloc[-1]
    v['price_change'] = v['current_price']/v['avg_cost']-1
    v['pl'] = v['current_price']*v['total_quantity']+v['total_dividends']-v['total_cost']
    v['pl_percent'] = (v['current_price']*v['total_quantity']+v['total_dividends'])/v['total_cost'] - 1
    print(k + ', pl: ' + str(v['pl']) + ',  pl_percent: ' + str(v['pl_percent']))

D05.SI, pl: 6121.390000000001,  pl_percent: 0.427931275302158
ME8U.SI, pl: -1016.3801907348625,  pl_percent: -0.06663398653492991
AJBU.SI, pl: -477.43979972839406,  pl_percent: -0.047616843002023956
C2PU.SI, pl: 27.950267028807502,  pl_percent: 0.0033536631405115536
BABA, pl: -677.9037778320308,  pl_percent: -0.030267853059524508
AAPL, pl: 1245.4802270507807,  pl_percent: 0.17587355359897372
MSFT, pl: 1772.409667968749,  pl_percent: 0.24957189275518155
NVDA, pl: 1820.8799999999992,  pl_percent: 0.18896402286397418
ARKK, pl: 2054.7797390747055,  pl_percent: 0.2081957613764791
SPY, pl: 1592.4902148437486,  pl_percent: 0.20933766271831034
ARKW, pl: 1213.830120849609,  pl_percent: 0.30697391642505134
ARKG, pl: 1653.269609375,  pl_percent: 0.1694496714437259
ARKQ, pl: 1070.0996484375,  pl_percent: 0.15068515205637922
ARKF, pl: 285.5399707031247,  pl_percent: 0.058423456691490516
FB, pl: 1221.9298022460944,  pl_percent: 0.17344811122979142
PRNT, pl: -1204.3204248046877,  pl_percent: -0.11768

In [22]:
splits = {}
for k, v in open_positions.items():
    value = stocks[k]['currency']
    if value not in splits:
        splits[value] = {}
    splits[value][k] = v

def calculate_total_pl(category):
    total_pl = 0
    for k, v in splits[category].items():
        total_pl += v['pl']
    return total_pl

# Attention
current_rate = 1.3345

print(calculate_total_pl('SGD'))
print(calculate_total_pl('USD')*current_rate)
print(calculate_total_pl('SGD') + calculate_total_pl('USD')*current_rate)

4655.520276565552
16267.575331821
20923.095608386553


In [23]:
from currency_exchange_transactions import currency_exchange_transactions

# Attention
remaining_usd_cash = 7222.51

total_original_sgd = 0
for cet in currency_exchange_transactions:
    total_original_sgd += cet['amount']
print(total_original_sgd)

usd_total = 0
for k, v in splits['USD'].items():
    usd_total += v['current_price']*v['total_quantity']
usd_total += remaining_usd_cash
print(usd_total)

from datetime import datetime, timedelta
now = datetime.now()
total_expense = 0

for t in transactions:
    dt = datetime.strptime(t['time'], '%Y-%m-%d')
    diff = now - dt
    if 'expense_ratio' in stocks[t['tick']]:        
        er = stocks[t['tick']]['expense_ratio']
        expense = t['quantity'] * er * open_positions[t['tick']]['current_price'] / 100
        total_expense += (expense * current_rate)
        
print(usd_total*current_rate)
print(usd_total*current_rate - total_original_sgd)
print(usd_total*current_rate - total_original_sgd + calculate_total_pl('SGD'))
print(total_original_sgd)
print((usd_total*current_rate - total_original_sgd + calculate_total_pl('SGD'))/total_original_sgd)
print(total_expense)
print(usd_total*current_rate - total_original_sgd + calculate_total_pl('SGD') - total_expense)
print((usd_total*current_rate - total_original_sgd + calculate_total_pl('SGD') - total_expense)/total_original_sgd)

211761.65625
168816.60923553468
225285.76502482104
13524.108774821041
18179.629051386593
211761.65625
0.08584948462021955
620.1134737166058
17559.515577669987
0.08292112882296172


In [7]:
# calculate ib account


In [8]:
# Simulation
from common import get_price

monthly_new_invest = 15000
start_td = datetime.strptime('2007-3-1', '%Y-%m-%d')

def simulate_stock(tick, start_td):
    print(tick)
    end_td = datetime.now()
    month_cnt = 0
    td = start_td
    d = load_data(tick)
    first_td = start_td = datetime.strptime(d['Date'].iloc[0], '%Y-%m-%d')
    if first_td > td:
        td = first_td

    simulated_transactions = []

    while td < end_td:
        price = get_price(d, td.strftime('%Y-%m-%d'))
        simulated_transactions.append({'date': td, 'price': price, 'quantity': round(monthly_new_invest/price)})
        month_cnt += 1
        td += timedelta(days=30.4)

    total_money_for_invest = monthly_new_invest * month_cnt
    print(total_money_for_invest)

    market_value = 0
    total_quantity = 0
    total_cost = 0
    for st in simulated_transactions:
        total_quantity += st['quantity']
        total_cost += st['quantity']*st['price']
    market_value = total_quantity * d['Adj Close'].iloc[-1]
    print(total_cost)
    print(market_value)
    print(market_value/total_cost-1)
    print((market_value/total_cost-1)/(month_cnt/12))

for s in splits['USD']:   
    simulate_stock(s, start_td)

BABA
1200000
1200201.1274490356
2239628.292907715
0.8660441501733356
0.12990662252600033
AAPL
2550000
2549955.8728220463
30569832.594451904
10.98837710105946
0.7756501483100795
MSFT
2550000
2550458.6509218216
20297043.799804688
6.958193634101322
0.4911666094659757
NVDA
2550000
2550905.2576036453
76219602.0
28.879432712291997
2.038548191455906
ARKK
1170000
1169938.9917545319
4951318.930664062
3.232117200606057
0.4972488000932395
SPY
2550000
2550366.8027648926
7583700.677490234
1.9735725344560735
0.13931100243219344
ARKW
1185000
1185285.2449588776
5956865.440368652
4.025680920017966
0.6114958359520961
ARKG
1170000
1169963.4877319336
4234271.754318237
2.6191486304642777
0.4029459431483504
ARKQ
1185000
1184907.3287315369
3792178.156311035
2.2004006257355377
0.33423806973198045
ARKF
405000
404916.7509441376
795190.4954910278
0.9638369952265387
0.42837199787846164
FB
1620000
1620080.759595871
6418451.186462402
2.96180940267662
0.3290899336307356
PRNT
855000
854980.5260734558
1436965.25239562